In [22]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd
import numpy as np
import time
import re
import sys
sys.path.append('C:/Users/J/Desktop/Git_Repositories/scraping_restaurant_nutrition')
from nutrient_finder_functions_w_nan import *

In [2]:
def parse_nut_info(nut_soup):
    # Getting the Right Soup
    # without using classes
    nut_space = nut_soup.findAll('basefont')[1].findAll('tr')
    # Item name
    item_name = nut_soup.find('h1').text
    # Extracting Variables
    serving_size = get_serving_size(nut_space[0])
    calories = get_calories(nut_space[0])
    total_fat = get_total_fat(nut_space[0])
    sat_fat = get_saturated_fat(nut_space[0])
    trans_fat = get_trans_fat(nut_space[0])
    cholesterol = get_cholesterol(nut_space[0])
    sodium = get_sodium(nut_space[0])
    potassium = get_potassium(nut_space[0])
    carb = get_total_carb(nut_space[0])
    fiber = get_dietary_fiber(nut_space[0])
    sugars = get_sugars(nut_space[0])
    protein = get_protein(nut_space[0])
    vit_a = get_vitamin_a(nut_soup)
    vit_c = get_vitamin_c(nut_soup)
    calcium = get_calcium(nut_soup)
    iron = get_iron(nut_soup)
    
    nutrient_df = pd.DataFrame([{
            'item_name':item_name,
            'serving_size':serving_size,
            'calories':calories,
            'fat_g':total_fat,
            'saturated_fat_g':sat_fat,
            'trans_fat_g':trans_fat,
            'cholesterol_mg':cholesterol,
            'sodium_mg':sodium,
            'potassium_mg':potassium,
            'carb_g':carb,
            'fiber_g':fiber,
            'sugar_g':sugars,
            'protein_g':protein,
            'vit_a_perc':vit_a,
            'vit_c_perc':vit_c,
            'calcium_perc':calcium,
            'iron_perc':iron
            }])

    return nutrient_df

In [9]:
nut_page = requests.get('http://www.dietfacts.com/html/nutrition-facts/dunkin-donuts-wake-up-wrap-with-bacon-bacon-scrambled62220.htm')

In [23]:
nut_soup = BeautifulSoup(nut_page.text, 'html.parser')
results = parse_nut_info(nut_soup)

In [27]:
def get_calcium(all_soup):
    tds = all_soup.findAll('td')
    for td in tds:
        font_text = td.find('font',attrs={'size':'2','face':'Verdana','color':'#666699' })
        if font_text == None:
            pass
        else:
            font_text = font_text.text.strip()
            if font_text.find('Calcium') != -1:
                    calcium = font_text.replace('\xa0', u'').strip()
                    calcium = calcium.replace('Calcium', u'').strip()
                    return calcium
    return np.nan

In [28]:
get_calcium(nut_soup)

'10%'

In [4]:
page = requests.get('http://www.dietfacts.com/list.asp?brand=Panera+Bread')
brand_soup = BeautifulSoup(page.text, 'html.parser')

In [10]:
def n_items(page_soup):
    tds = page_soup.findAll('td')
    for td in tds:
        bs = td.findAll('b')
        if bs != None:
            for b in bs:
                if b.text.strip().find('Page 1 of') !=-1:
                    page_ind = b.text.strip()
                    page_ind = page_ind.replace('Page 1 of ',u'').strip()
                    return page_ind
                
    return "not found"

In [12]:
n_pages = n_items(brand_soup)

In [18]:
page_url = 'http://www.dietfacts.com/list.asp?brand=Panera+Bread'

In [23]:
page_links = []
for i in range(2,int(n_pages)+1):
    page_links.append(page_url+'&page='+str(i))

In [24]:
page_links

['http://www.dietfacts.com/list.asp?brand=Panera+Bread&page=2',
 'http://www.dietfacts.com/list.asp?brand=Panera+Bread&page=3',
 'http://www.dietfacts.com/list.asp?brand=Panera+Bread&page=4',
 'http://www.dietfacts.com/list.asp?brand=Panera+Bread&page=5',
 'http://www.dietfacts.com/list.asp?brand=Panera+Bread&page=6',
 'http://www.dietfacts.com/list.asp?brand=Panera+Bread&page=7',
 'http://www.dietfacts.com/list.asp?brand=Panera+Bread&page=8',
 'http://www.dietfacts.com/list.asp?brand=Panera+Bread&page=9']

In [43]:
base_url = 'http://www.dietfacts.com/'

In [50]:
a_links = brand_soup.findAll('table')[2].findAll('a')
rest_links = []
rest_food_names = []
rest_food_desc = []
food_df = pd.DataFrame()
for a in a_links:
    href_res = a['href']
    if href_res.find('html/nutrition-facts')!= -1:
        if a.find('b') != None:
            food_url = base_url+a['href']
            nut_page = requests.get(food_url)
            nut_soup = BeautifulSoup(nut_page.text, 'html.parser')
            nut_df = parse_nut_info(nut_soup)
            food_df = food_df.append(nut_df, ignore_index=True)


In [58]:
food_df.drop_duplicates

<bound method DataFrame.drop_duplicates of    calcium_perc calories carb_g cholesterol_mg fat_g fiber_g  iron_perc  \
0     not found      260    34g           30mg   11g      1g  not found   
1     not found      510    50g           85mg   30g      2g  not found   
2     not found      390    40g            0mg   22g      1g  not found   
3     not found      120    25g            0mg    0g      1g  not found   
4     not found      120    25g            0mg    0g      1g  not found   
5     not found      120    25g            0mg    0g      1g  not found   
6     not found      110    23g            0mg    0g     <1g  not found   
7     not found      110    23g            0mg    0g     <1g  not found   
8     not found      140    26g            0mg    2g      1g  not found   
9     not found      140    26g            0mg    2g      1g  not found   
10    not found      120    24g            0mg  0.5g      1g  not found   
11    not found      120    24g            0mg  0.5g     